In [28]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("secrets.json")
client = bigquery.Client(credentials=credentials)
dataset = client.get_dataset('bigquery-public-data.openaq')


In [29]:
for table_item in list(client.list_tables(dataset)):
    print(table_item.table_id)

global_air_quality


In [30]:
table_ref = dataset.table("global_air_quality")
table = client.get_table(table_ref)
'''
    Each SchemaField tells us about a specific column
    (which we also refer to as a field).
    In order, the information is:

    - The name of the column
    - The field type (or datatype) in the column
    - The mode of the column ('NULLABLE' means that a column allows NULL values, and is the default)
    - A description of the data in that column
'''
print(table.schema)


[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', (), None), SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', (), None), SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', (), None), SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', (), None), SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', (), None), SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', (), None), SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', (), None), SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', (), None), SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', (), None), SchemaField('lon

In [31]:
df = client.list_rows(table, max_results=5).to_dataframe()
df

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


---

### Queries

In [32]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
    SELECT city
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
"""
query_job = client.query(query)
us_cities = query_job.to_dataframe()
us_cities

,city
0,BROWN
1,BROWN
2,BROWN
3,BROWN
4,Houston
...,...
3718,Atlanta
3719,Mammoth Cave NP
3720,Las Cruces
3721,MARTIN


In [ ]:
# You can select multiple columns with comma-separated names
query = """
    SELECT city, country
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
"""
# Or use * to select all columns
query = """
    SELECT *
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
"""

---

### Group By, Having & Count

Now that you can select raw data, you're ready to learn **how to group** your data and **count things** within those groups.

This can help you answer questions like:

- How many of each kind of fruit has our store sold?
- How many species of animal has the vet office treated?

##### COUNT
- Returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.
- COUNT() is an example of an aggregate function, which takes many values and returns one.

##### GROUP BY
- It takes the name of one or more columns, and **treats all rows with the same value in that column as a single group** when you apply aggregate functions


##### GROUP BY ... HAVING
- HAVING is used in combination with GROUP BY to ignore groups that don't meet certain criteria.

Here is an example. Suppose this table:

|  ID   |        Name        | Animal |
| :---: | :----------------: | :----: |
|   1   | Dr. Harris Bonkers | Rabbit |
|   2   |        Moon        |  Dog   |
|   3   |       Ripley       |  Cat   |
|   4   |        Tom         |  Cat   |

In [ ]:
# First, you can SELECT directly from a COUNT sentence:
QUERY = """
    SELECT COUNT(ID)
    FROM 'bigquery-public-data.pet_records.pets'
"""
# It's going to return something like this
response = {
    'f0_': 4
}

In [ ]:
# And here is an example with all techniques together

# For example, say we want to know how many of each type of animal
# we have in the pets table.

# We can use GROUP BY to group together rows that have the same
# value in the Animal column, while using COUNT() to find out
# how many ID's we have in each group with value > 1 using HAVING.

QUERY = """
    SELECT Animal COUNT(ID)
    FROM 'bigquery-public-data.pet_records.pets'
    GROUP BY Animal
    HAVING COUNT(ID) > 1
"""
# Other animals have just 1 records.
response = {
    'Animal': 'Cat',
    'f0_': 2
}